# Week 2 Exercise — Stayez AI Booking Assistant (Chiku)

**Student:** Vagz1216  
**Course:** LLM Engineering — Andela AI Engineering Bootcamp  
**Exercise:** Week 2 End-of-Week Exercise  

---

## Project Overview

For my Week 2 exercise I built **Chiku**, an AI customer assistant for [Stayez](https://stayez.co.ke) — a real Kenyan travel booking platform that I work on. Chiku helps guests discover and book Airbnbs, local experiences, and services across Kenya.

This project goes beyond a generic Q&A bot by applying all Week 2 concepts to a **real-world commercial use case**.

### Week 2 Requirements Covered

| Requirement | Implementation |
|---|---|
| **Gradio UI** | `gr.ChatInterface` with green Stayez theme |
| **Streaming** | Word-by-word streaming via `yield` |
| **System Prompt / Expertise** | Deep Stayez persona — knows all listings, prices, cities, contact info |
| **Tool Use (Bonus)** | `search_properties()` and `get_property_details()` tools for live data lookup |
| **Model Switching** | Groq Llama, Gemini 1.5 Flash, Liquid Thinking, Claude Sonnet |

### Models Used
- **Groq Llama 3.3 70B** (default, blazing fast, free) — via Groq native endpoint
- **Gemini 1.5 Flash** (smart, free) — via Google native endpoint
- **Liquid LFM 2.5 Thinking** (reasoning model, free) — via OpenRouter
- **Claude 3.5 Sonnet** (premium) — via OpenRouter with `max_tokens` cap

### Real-World Application
This is an MVP I will continue building after the course using:
- WooCommerce REST API for live product listings
- RAG (Week 4/5) for personalized recommendations
- WhatsApp deployment via the Texty plugin already on the site

In [ ]:
# Cell 1: Imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Cell 2: Load API Keys and Initialize Clients

load_dotenv(override=True)

groq_api_key       = os.getenv('GROQ_API_KEY')
gemini_api_key     = os.getenv('GEMINI_API_KEY')   # Use GEMINI key (higher quota than GOOGLE_API_KEY)
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

for name, key in [
    ("Groq",       groq_api_key),
    ("Gemini",     gemini_api_key),
    ("OpenRouter", openrouter_api_key),
]:
    if key:
        print(f"{name} API Key exists and begins {key[:6]}")
    else:
        print(f"{name} API Key NOT set!")

# Groq — native, fastest
groq = OpenAI(api_key=groq_api_key,
              base_url="https://api.groq.com/openai/v1")

# Gemini — native Google endpoint, using GEMINI_API_KEY for better quota
gemini = OpenAI(api_key=gemini_api_key,
                base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

# OpenRouter — used for Claude AND Liquid (both work reliably here)
openrouter = OpenAI(api_key=openrouter_api_key,
                    base_url="https://openrouter.ai/api/v1")

# Model IDs
GROQ_MODEL    = "llama-3.3-70b-versatile"               # via groq
MODEL_GEMINI  = "gemini-1.5-flash"                      # via gemini (higher free quota than 2.0-flash!)
MODEL_CLAUDE  = "anthropic/claude-3-5-sonnet-20241022"  # via openrouter (handles Anthropic compat!)
MODEL_LIQUID  = "liquid/lfm-2.5-1.2b-thinking:free"    # via openrouter

CLAUDE_MAX_TOKENS = 1024

print("\nAll clients ready!")
print(f"  Groq   → llama-3.3-70b-versatile (native)")
print(f"  Gemini → gemini-1.5-flash (native Google)")
print(f"  Claude → claude-3-5-sonnet via OpenRouter")
print(f"  Liquid → liquid/lfm-2.5 via OpenRouter")

In [ ]:
# Cell 3: Stayez Knowledge Base
# All the real listings, prices, cities and booking links scraped from stayez.co.ke

STAYEZ_LISTINGS = {
    "the hive studio air bnb": {
        "name": "The Hive Studio Air BnB",
        "price_per_night": 2500,
        "city": "Nairobi",
        "type": "home",
        "bedrooms": 0,  # Studio
        "description": "A cozy studio apartment, perfect for solo travelers or couples. Compact, modern and well-equipped.",
        "url": "https://stayez.co.ke/product/the-hive-studio-air-bnb/"
    },
    "modern 1 br kilimani": {
        "name": "Modern 1 BR Kilimani",
        "price_per_night": 5000,
        "city": "Nairobi",
        "type": "home",
        "bedrooms": 1,
        "description": "A sleek, modern 1-bedroom apartment in the heart of Kilimani, Nairobi. Great for business travelers and couples.",
        "url": "https://stayez.co.ke/product/modern-1-br-kilimani/"
    },
    "pure 1br goldpark": {
        "name": "Pure 1BR Goldpark",
        "price_per_night": 5500,
        "city": "Nairobi",
        "type": "home",
        "bedrooms": 1,
        "description": "A clean one-bedroom unit in Goldpark estate. Quiet, safe neighborhood with 24/7 security.",
        "url": "https://stayez.co.ke/product/pure-1br-goldpark/"
    },
    "padmore residence": {
        "name": "Padmore Residence",
        "price_per_night": 5500,
        "city": "Nairobi",
        "type": "home",
        "bedrooms": 1,
        "description": "An elegant, well-furnished residence in a prestigious estate. Great amenities and modern interior design.",
        "url": "https://stayez.co.ke/product/padmore-residence/"
    },
    "pure himalayas": {
        "name": "Pure Himalayas",
        "price_per_night": 5000,
        "city": "Nairobi",
        "type": "home",
        "bedrooms": 1,
        "description": "A serene one-bedroom unit inspired by calm mountain aesthetics. Perfect for a peaceful retreat in Nairobi.",
        "url": "https://stayez.co.ke/product/pure-himalayas/"
    },
    "golden mango": {
        "name": "Golden Mango",
        "price_per_night": 6500,
        "city": "Nairobi",
        "type": "home",
        "bedrooms": 1,
        "description": "A vibrant, well-curated apartment with warm tropical tones and great city views. Very popular with solo and couple guests.",
        "url": "https://stayez.co.ke/product/golden-mango/"
    },
    "smart1": {
        "name": "Smart1",
        "price_per_night": 7500,
        "city": "Nairobi",
        "type": "home",
        "bedrooms": 1,
        "description": "A smart-home enabled, premium apartment packed with tech-forward amenities. Ideal for remote workers and tech enthusiasts.",
        "url": "https://stayez.co.ke/product/smart1/"
    },
    "pure golden mango 2br": {
        "name": "Pure Golden Mango 2BR",
        "price_per_night": 8000,
        "city": "Nairobi",
        "type": "home",
        "bedrooms": 2,
        "description": "A spacious 2-bedroom version of the popular Golden Mango. Great for small families or groups of friends.",
        "url": "https://stayez.co.ke/product/pure-golden-mango-2br/"
    },
    "pure 3br goldpark": {
        "name": "Pure 3BR Goldpark",
        "price_per_night": 9500,
        "city": "Nairobi",
        "type": "home",
        "bedrooms": 3,
        "description": "A large 3-bedroom apartment in the Goldpark estate. Ideal for families, large groups or extended stays.",
        "url": "https://stayez.co.ke/product/pure-3br-goldpark/"
    },
    "longonot hike": {
        "name": "Longonot Hike",
        "price_per_night": 2500,
        "city": "Nakuru",
        "type": "experience",
        "bedrooms": None,
        "description": "A thrilling guided hike up Mount Longonot volcano in the Great Rift Valley. Spectacular crater views await!",
        "url": "https://stayez.co.ke/product/longonot-hike/"
    },
    "dancing classes": {
        "name": "Dancing Classes",
        "price_per_night": 1500,
        "city": "Nairobi",
        "type": "experience",
        "bedrooms": None,
        "description": "Fun and engaging dance sessions in Nairobi. Great for solo travelers looking to connect with local culture.",
        "url": "https://stayez.co.ke/product/dancing-classes/"
    },
    "service product": {
        "name": "Service Product",
        "price_per_night": 1500,
        "city": "Nairobi",
        "type": "service",
        "bedrooms": None,
        "description": "A range of personalized guest services available to enhance your stay — from airport pickups to in-stay concierge.",
        "url": "https://stayez.co.ke/product/service-product/"
    }
}

print(f"Stayez knowledge base loaded: {len(STAYEZ_LISTINGS)} listings")

In [ ]:
# Cell 4: System Prompt — The Stayez Expert Persona
# This is what makes the AI knowledgeable and on-brand

SYSTEM_PROMPT = """
You are Chiku, a warm and knowledgeable AI customer assistant for Stayez — a premium Kenyan travel booking platform.
Stayez offers three categories of bookings:
  - HOMES: Curated Airbnbs ranging from cozy studios to spacious 3-bedroom apartments
  - EXPERIENCES: Guided local adventures like hikes and cultural activities  
  - SERVICES: Personal guest services like airport pickups and in-stay concierge

CITIES AVAILABLE: Nairobi, Mombasa, Nakuru, Kisumu, Diani

YOUR PERSONALITY:
- Warm, friendly and genuinely helpful like a knowledgeable local friend
- Enthusiastic about Kenya and honest about what each listing offers
- Concise but thorough — give helpful answers without overwhelming the guest
- Always guide the guest toward making a booking decision

BOOKING PROCESS:
- Guests can browse homes at: https://stayez.co.ke/shop
- Guests can browse experiences at: https://stayez.co.ke/experiences
- Guests can browse services at: https://stayez.co.ke/services
- All prices shown are per night in Kenyan Shillings (KSh)
- For questions about availability or complex bookings, guests should contact: info@stayez.co.ke or +254 710 428 119
- Stayez also has WhatsApp: https://wa.me/+254710428119

TOOLS AT YOUR DISPOSAL:
- Use search_properties when the guest wants to find listings by city or budget
- Use get_property_details when the guest asks about a specific property by name

IMPORTANT:
- Always present prices in KSh format (e.g. KSh 5,000/night)
- When you find a relevant property, always share the booking link!
- If asked something you don't know, direct the guest to contact Stayez directly
- Do NOT make up properties or prices that you have not been given
"""

In [ ]:
# Cell 5: Tool Functions — The Real Python Logic
# These are the functions the LLM will be able to call

def search_properties(city=None, max_budget=None, property_type=None, min_bedrooms=None):
    """Search Stayez listings by city, budget, type, or number of bedrooms."""
    print(f"TOOL CALLED: search_properties(city={city}, max_budget={max_budget}, type={property_type}, min_bedrooms={min_bedrooms})")
    
    results = []
    for key, listing in STAYEZ_LISTINGS.items():
        # Filter by city
        if city and listing["city"].lower() != city.lower():
            continue
        # Filter by budget
        if max_budget and listing["price_per_night"] > max_budget:
            continue
        # Filter by type
        if property_type and listing["type"].lower() != property_type.lower():
            continue
        # Filter by bedrooms
        if min_bedrooms is not None and listing["bedrooms"] is not None:
            if listing["bedrooms"] < min_bedrooms:
                continue
        results.append(listing)
    
    if not results:
        return "No listings found matching those criteria. Try adjusting your filters, or contact us at info@stayez.co.ke for personalized recommendations."
    
    # Format for the LLM
    output = f"Found {len(results)} listing(s):\n\n"
    for r in results:
        beds = f"{r['bedrooms']} BR" if r['bedrooms'] else "Studio/N/A"
        output += f"{r['name']}** — KSh {r['price_per_night']:,}/night | {r['city']} | {beds}\n"
        output += f"  {r['description']}\n"
        output += f"  {r['url']}\n\n"
    return output


def get_property_details(property_name):
    """Get full details and booking link for a specific Stayez property."""
    print(f"TOOL CALLED: get_property_details(property_name={property_name})")
    
    # Match by lowercase key
    key = property_name.lower().strip()
    listing = STAYEZ_LISTINGS.get(key)
    
    # Fuzzy match if exact key not found
    if not listing:
        for k, v in STAYEZ_LISTINGS.items():
            if key in k or k in key:
                listing = v
                break
    
    if not listing:
        return f"Property '{property_name}' not found. Available properties are: {', '.join([v['name'] for v in STAYEZ_LISTINGS.values()])}"
    
    beds = f"{listing['bedrooms']} Bedroom(s)" if listing['bedrooms'] else "Studio / Not applicable"
    return (
        f"**{listing['name']}**\n"
        f"Type: {listing['type'].title()}\n"
        f"City: {listing['city']}\n"
        f"Size: {beds}\n"
        f"Price: KSh {listing['price_per_night']:,} per night\n"
        f"About: {listing['description']}\n"
        f"Book here: {listing['url']}"
    )


# Test both tools
print("--- Testing search_properties ---")
print(search_properties(city="Nairobi", max_budget=6000))

print("--- Testing get_property_details ---")
print(get_property_details("smart1"))

In [ ]:
# Cell 6: Tool Schema Definitions
# We describe our Python functions in JSON-Schema so the LLM knows when and how to call them

search_function = {
    "name": "search_properties",
    "description": "Search Stayez listings by city, budget, property type, or number of bedrooms. Use this when a guest wants to browse or filter available stays, experiences or services.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to search in. Options: Nairobi, Mombasa, Nakuru, Kisumu, Diani"
            },
            "max_budget": {
                "type": "number",
                "description": "Maximum price per night in Kenyan Shillings (KSh)"
            },
            "property_type": {
                "type": "string",
                "description": "Type of listing. Options: home, experience, service"
            },
            "min_bedrooms": {
                "type": "number",
                "description": "Minimum number of bedrooms required (use 0 for studio)"
            }
        },
        "required": [],
        "additionalProperties": False
    }
}

details_function = {
    "name": "get_property_details",
    "description": "Get full details and direct booking link for a specific Stayez property by name. Use this when the guest asks about a specific listing they have already seen or heard of.",
    "parameters": {
        "type": "object",
        "properties": {
            "property_name": {
                "type": "string",
                "description": "The exact or approximate name of the property (e.g. 'Smart1', 'Pure 3BR Goldpark', 'Longonot Hike')"
            }
        },
        "required": ["property_name"],
        "additionalProperties": False
    }
}

tools = [
    {"type": "function", "function": search_function},
    {"type": "function", "function": details_function}
]

print(f"Registered {len(tools)} tools: {[t['function']['name'] for t in tools]}")

In [ ]:
# Cell 7: Tool Call Handler
# Routes the LLM's tool request to the correct Python function

def handle_tool_calls(message):
    """Execute all tool calls the LLM requested and return the results."""
    responses = []
    for tool_call in message.tool_calls:
        fn_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        if fn_name == "search_properties":
            result = search_properties(**arguments)
        elif fn_name == "get_property_details":
            result = get_property_details(**arguments)
        else:
            result = f"Unknown tool: {fn_name}"
        
        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id
        })
    return responses

In [ ]:
# Cell 8: Chat Function with Streaming + Tool Calling + Model Switching

MODEL_LABEL_GROQ    = "Groq Llama (Fast)"
MODEL_LABEL_GEMINI  = "Gemini 1.5 Flash (Smart)"
MODEL_LABEL_LIQUID  = "Liquid Thinking (Reasoning)"
MODEL_LABEL_CLAUDE  = "Claude Sonnet (Premium)"

# Gemini and Groq and Claude support tool calling reliably
# Liquid does NOT support tools
TOOL_CAPABLE_MODELS = {MODEL_LABEL_GROQ, MODEL_LABEL_GEMINI, MODEL_LABEL_CLAUDE}

def chat(message, history, selected_model):
    history_formatted = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history_formatted + [{"role": "user", "content": message}]

    extra_params = {}
    if selected_model == MODEL_LABEL_GROQ:
        client = groq
        model  = GROQ_MODEL

    elif selected_model == MODEL_LABEL_GEMINI:
        client = gemini
        model  = MODEL_GEMINI

    elif selected_model == MODEL_LABEL_CLAUDE:
        client = openrouter          # OpenRouter correctly proxies Anthropic API!
        model  = MODEL_CLAUDE
        extra_params["max_tokens"] = CLAUDE_MAX_TOKENS

    else:  # Liquid — OpenRouter only, no tools
        client = openrouter
        model  = MODEL_LIQUID

    supports_tools = selected_model in TOOL_CAPABLE_MODELS

    try:
        if supports_tools:
            response = client.chat.completions.create(
                model=model, messages=messages, tools=tools, **extra_params
            )
            while response.choices[0].finish_reason == "tool_calls":
                tool_message   = response.choices[0].message
                tool_responses = handle_tool_calls(tool_message)
                messages.append(tool_message)
                messages.extend(tool_responses)
                response = client.chat.completions.create(
                    model=model, messages=messages, tools=tools, **extra_params
                )

        stream = client.chat.completions.create(
            model=model, messages=messages, stream=True, **extra_params
        )
        result = ""
        for chunk in stream:
            result += chunk.choices[0].delta.content or ""
            yield result

    except Exception as e:
        err = str(e)
        if "429" in err or "quota" in err.lower() or "rate" in err.lower():
            yield f"Rate limit hit on {selected_model}.** Please wait a moment and try again, or switch to Groq Llama which has no rate limits!"
        elif "402" in err or "credit" in err.lower():
            yield f"Insufficient credits for {selected_model}.** Try switching to Groq Llama (always free) or Gemini instead!"
        elif "404" in err or "not_found" in err.lower():
            yield f"Model not found: {selected_model}.** Please try a different model from the dropdown."
        else:
            yield f"Unexpected error:** {err[:300]}\n\nTry switching models or refreshing."

In [ ]:
# Cell 9: Launch the Stayez Gradio Chat Assistant!

model_selector = gr.Dropdown(
    choices=[
        MODEL_LABEL_GROQ,
        MODEL_LABEL_GEMINI,
        MODEL_LABEL_LIQUID,
        MODEL_LABEL_CLAUDE,
    ],
    value=MODEL_LABEL_GROQ,
    label="Choose your AI model",
    info="Groq is always free & fastest. Gemini, Claude & Liquid also available."
)

demo = gr.ChatInterface(
    fn=chat,
    type="messages",
    title="Stayez AI Assistant — Chiku",
    description=(
        "Welcome to Stayez! I'm **Chiku**, your personal travel assistant. "
        "Ask me about homes, experiences, and services across Kenya. "
        "I can search by city, budget, or property size!"
    ),
    additional_inputs=[model_selector],
    examples=[
        ["What affordable stays do you have in Nairobi under KSh 6,000?", MODEL_LABEL_GROQ],
        ["I need a 3-bedroom place for a family of 5", MODEL_LABEL_GROQ],
        ["Tell me more about the Smart1 apartment", MODEL_LABEL_GEMINI],
        ["What experiences can I do near Nakuru?", MODEL_LABEL_LIQUID],
        ["I want to plan a romantic weekend for 2 in Nairobi — what do you recommend?", MODEL_LABEL_CLAUDE]
    ],
    theme=gr.themes.Soft(primary_hue="green"),
    flagging_mode="never"
)

demo.launch(inbrowser=True)

---

## 🚀 What's Next — Post-Course Upgrade Path

Once you have completed the LLM Engineering course, here is how you evolve this MVP into a full production assistant:

| Phase | Feature | Technology |
|---|---|---|
| **Phase 2** | Live product listings from WooCommerce | WooCommerce REST API → replaces hardcoded dict |
| **Phase 3** | Real-time availability checking | MWB Bookings plugin API |
| **Phase 4** | Actually complete a booking via chat | WooCommerce Order API |
| **Phase 5** | WhatsApp chatbot deployment | Twilio or WhatsApp Cloud API + Texty plugin |
| **Phase 6** | Personalized recommendations (RAG) | Vector DB (Week 4/5 of this course!) |
| **Phase 7** | Guest memory (returning users) | Persistent embeddings + user profiles |

**Contact Info embedded in the assistant:**
- 📧 info@stayez.co.ke
- 📞 +254 710 428 119
- 💬 https://wa.me/+254710428119